# Sea Surface Temperature

In [ ]:
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cmocean
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from OHC import t2da, t2ds
from SST import SST_index, EOF_SST_analysis
from maps import map_robinson, map_eq_earth
from grid import find_array_idx
from paths import path_results, path_samoc, file_ex_ocn_ctrl, file_ex_ocn_rect
from regions import boolean_mask, global_ocean, gl_ocean_rect
from plotting import shifted_color_map, discrete_cmap
from timeseries import IterateOutputCESM
from xr_DataArrays import xr_AREA
from xr_regression import xr_linear_trends_2D, xr_linear_trend, ocn_field_regression

# global yearly avg. SST map and trends

In [ ]:
MASK = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False).REGION_MASK

In [ ]:
# %%time
# # ca. 4:30 min for ctrl/rcp, 1:25 for lpi
# # stacking files into one xr Dataset object
# for run in ['ctrl', 'rcp']:  # ['lpi', 'lpd']:
#     for i, (y,m,s) in enumerate(IterateOutputCESM('ocn', run, 'yrly', name='TEMP_PD')):
#         print(y)
#         da = xr.open_dataset(s, decode_times=False).TEMP[0,:,:]
#         da = da.drop(['z_t', 'ULONG', 'ULAT'])
#         da['TLAT' ] = da['TLAT' ].round(decimals=2)
#         da['TLONG'] = da['TLONG'].round(decimals=2)
#         del da.encoding["contiguous"]
#         ds = t2ds(da=da, name='SST', t=int(round(da.time.item())))
#         ds.to_netcdf(path=f'{path_samoc}/SST/SST_yrly_{run}_{y}.nc', mode='w')
     
#     combined = xr.open_mfdataset(f'{path_samoc}/SST/SST_yrly_{run}_*.nc',
#                                  concat_dim='time',
#                                  autoclose=True,
#                                  coords='minimal')
#     combined.to_netcdf(f'{path_samoc}/SST/SST_yrly_{run}.nc')
#     # remove extra netCDF files

In [ ]:
SST_yrly_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_ctrl.nc')
SST_yrly_rcp  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_rcp.nc' )
SST_yrly_lpd  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_lpd.nc' )
SST_yrly_lpi  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_lpi.nc' )

In [ ]:
# mean CTRL SST
xa    = SST_yrly_ctrl.mean(dim='time').where(MASK>0)
fn    = f'{path_results}/SST/SST_ctrl_mean'
cm    = discrete_cmap(17, cmocean.cm.thermal)
f, ax = map_robinson(xa=xa, domain='ocn_U', cmap=cm, minv=-2, maxv=32, label='SST [$^\circ$C]', filename=fn)

In [ ]:
%%time
# 1:23 min
SST_trend_ctrl = ocn_field_regression(SST_yrly_ctrl)
SST_trend_rcp  = ocn_field_regression(SST_yrly_rcp )

In [ ]:
cmap  = discrete_cmap(12, shifted_color_map(cmocean.cm.balance, start=.25, midpoint=0.5, stop=1., name='shrunk'))

xa    = SST_trend_ctrl*100
fn    = f'{path_results}/SST/SST_trend_ctrl'
label = '100-299 SST trend [K/century]'
f, ax = map_robinson(xa=xa, domain='ocn_T', cmap=cmap, minv=-2, maxv=4, label=label, filename=fn)

xa    = SST_trend_rcp*100
fn    = f'{path_results}/SST/SST_trend_rcp'
label = '2000-2099 SST trend [K/century]'
f, ax = map_robinson(xa=xa, domain='ocn_T', cmap=cmap, minv=-2, maxv=4, label=label, filename=fn)

### hiatus trend

In [ ]:
SST_yrly_rcp[20:29,:,:]

In [ ]:
SST_trend_rcp_hiatus  = ocn_field_regression(SST_yrly_rcp[20:29,:,:])

In [ ]:
fn = f'{path_results}/SST/SST_trend_rcp_hiatus'
cmap = shifted_color_map(cmocean.cm.balance, start=.25, midpoint=0.5, stop=1., name='shrunk')
label = 'SST trend [K/century]'
xa = SST_trend_rcp_hiatus*100
f, ax = map_robinson(xa=xa, domain='ocn_T', cmap=cmap, minv=-4, maxv=8, label=label, filename=fn)

# low resolution runs

In [ ]:
from paths import file_ex_ocn_lpd, file_ex_ocn_lpi

In [ ]:
REGION_MASK = xr.open_dataset(file_ex_ocn_lpd, decode_times=False).REGION_MASK

In [ ]:
ds = xr.open_dataset(file_ex_ocn_lpd, decode_times=False)

In [ ]:
ds.TAREA.plot()

In [ ]:
SST_pd = xr.open_dataset(file_ex_ocn_lpd, decode_times=False).TEMP[0,0,:,:]
SST_pi = xr.open_dataset(file_ex_ocn_lpi, decode_times=False).TEMP[0,0,:,:]

In [ ]:
REGION_MASK.plot()

In [ ]:
map_eq_earth(domain='ocn_T', xa=SST_pd.where(SST_pd.TLAT>0).where(SST_pd.TLAT<60).where(SST_pd.TLONG>280), cmap=cmocean.cm.thermal, maxv=30, minv=5, label='')

In [ ]:
SST_pd.TLAT.where(REGION_MASK==6).where(SST_pd.TLAT>0).plot()

In [ ]:
(SST_pd-SST_pi).plot()

In [ ]:
SST_pi.TLONG.plot()

In [ ]:
SST_new = xr.concat([SST_pd, SST_pi], dim='time')

In [ ]:
map_eq_earth(domain='ocn_T', xa=SST_pd, cmap=cmocean.cm.thermal, maxv=30, minv=-2, label='')

In [ ]:
# SST_pd.to_netcdf('SST1.nc')
# SST_new.to_netcdf('SST2.nc')
# # 500kB extra for an extra 2D ocn field